<a href="https://colab.research.google.com/github/linhvien/Stock-analysis/blob/main/Vietnamese_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests

import re
import json
import csv
from io import StringIO
from datetime import datetime, timedelta
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Overview

In [2]:
start_date = '2019-01-01'
end_date = str(datetime.now().strftime('%Y-%m-%d'))
stock = 'VIC'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

#Financial data
Loading quarterly financial data of Vietnamese stocks from api vndirect.

In [3]:
class Financialdata:
  url_BS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=1,89,101,411&fromDate={}&toDate={}'
  url_IS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=2,90,102,412&fromDate={}&toDate={}'
  url_CF = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=3,91,103,413&fromDate={}&toDate={}'
  url_ratio = 'https://finfo-api.vndirect.com.vn/v4/ratios?q=code:{}~itemCode:53030,52005,51050,53021,52001,52002,54018,712010,712020,712030,712040~reportDate:{}'
  
  def __init__(self,stock,start_date,end_date,*arg, **karg):
    self.stock = stock
    self.start_date = start_date
    self.end_date = end_date
    self.session = requests.Session()

# Get balance sheet
  def get_BS(self):
    try:
      page = self.session.get(self.url_BS.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

# get income statement
  def get_IS(self):
    try:
      page = self.session.get(self.url_IS.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

# get cashflow statement
  def get_CF(self):
    try:
      page = self.session.get(self.url_CF.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

In [4]:
BS_data = Financialdata(stock,start_date,end_date).get_BS()
BS_data.to_excel(r'BS_{}.xlsx'.format(stock),index=True)
BS_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,,
Tiền và các khoản tương đương tiền,23302535000000,23042288000000,28175943000000,30288635000000,26969148000000,10999921000000,18488606000000,16526657094574,23549297107100,7441890315078
Tiền,11299479000000,9679176000000,9085433000000,9922420000000,9907075000000,7161118000000,7638488000000,9910083810686,13277188937271,6028456570925
Các khoản tương đương tiền,12003056000000,13363112000000,19090510000000,20366215000000,17062073000000,3838803000000,10850118000000,6616573283888,10272108169829,1413433744153
Các khoản đầu tư tài chính ngắn hạn,10327516000000,5969254000000,11763511000000,4987739000000,870606000000,9483443000000,10419573000000,879925574841,1050958199431,1845851929698
Đầu tư ngắn hạn,1766496000000,2290664000000,6027042000000,816435000000,0,8582371000000,9539371000000,0,0,0


In [5]:
IS_data = Financialdata(stock,start_date,end_date).get_IS()
IS_data.to_excel(r'IS_{}.xlsx'.format(stock),index=True)
IS_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,
Các khoản giảm trừ doanh thu,38303000000,11037000000,102026000000,76099000000,-6304000000,288536000000,10871219860,21503540891,92030956552
Doanh thu thuần,37442438000000,23294421000000,35821395000000,35913718000000,23358773000000,38175708000000,31570918174253,39220453866772,21822679632050
Giá vốn hàng bán,26986417000000,20008967000000,30241372000000,29612674000000,19944575000000,29315897000000,20290596055885,26443132177439,17043520693937
Lợi nhuận gộp,10456021000000,3285454000000,5580023000000,6301044000000,3414198000000,8859811000000,11280322118368,12777321689333,4779158938113
Doanh thu hoạt động tài chính,3917045000000,7128520000000,11780480000000,4693043000000,6744100000000,8611984000000,1013408932129,887264230744,3439230133986


In [6]:
CF_data = Financialdata(stock,start_date,end_date).get_CF()
CF_data.to_excel(r'CF_{}.xlsx'.format(stock),index=True)
CF_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,
Chi phí khấu hao tài sản cố định,4093107000000,2931384000000,2835598000000,2832365000000,1973937000000,3120229000000,2060985863151,2120346898774,1476839234519
Phân bổ lợi thế thương mại,0,0,0,0,0,0,0,0,0
"Dự phòng giảm giá các khoản đầu tư ngắn hạn, dài hạn",3564340000000,133562000000,-861663000000,-870812000000,1477219000000,1414621000000,-599743206070,1269396920822,-332571028167
"Lãi, lỗ chênh lệch tỷ giá hối đoái chưa thực hiện",73244000000,-130009000000,-585577000000,-170054000000,-426937000000,32528000000,-192863826881,164344127544,1374347097
Lãi/(lỗ) từ thanh lý tài sản cố định,0,0,0,0,0,0,0,0,0


#Full stock and industry

In [7]:
# Get industry list from vietstock
r = requests.get('https://finance.vietstock.vn/chi-so-nganh.htm',headers=headers)
soup = bs(r.content)
def Getsectorname():
  sectors = soup.select('a[href^="/nganh"]')
  all_sector = []
  for i in sectors:
    sector_name = i.get('title')
    if sector_name not in all_sector:
      all_sector.append(sector_name)
  return all_sector

In [8]:
Getsectorname()

['Bán buôn',
 'Bảo hiểm',
 'Bất động sản',
 'Chứng khoán',
 'Công nghệ và thông tin',
 'Bán lẻ',
 'Chăm sóc sức khỏe',
 'Khai khoáng',
 'Ngân hàng',
 'Nông - Lâm - Ngư',
 'SX Thiết bị, máy móc',
 'SX Hàng gia dụng',
 'Sản phẩm cao su',
 'SX Nhựa - Hóa chất',
 'Thực phẩm - Đồ uống',
 'Chế biến Thủy sản',
 'Vật liệu xây dựng',
 'Tiện ích',
 'Vận tải - kho bãi',
 'Xây dựng',
 'Dịch vụ lưu trú, ăn uống, giải trí',
 'SX Phụ trợ',
 'Thiết bị điện',
 'Dịch vụ tư vấn, hỗ trợ',
 'Tài chính khác']

In [9]:
def GetURL():
    sectors = soup.select('a[href^="/nganh"]')
    all_sector_URL = []
    for sector in sectors:
        sector_ID = sector.get('href')
        sector_URL = 'https://finance.vietstock.vn' + sector_ID
        if sector_URL not in all_sector_URL:
            all_sector_URL.append(sector_URL)
    return all_sector_URL

In [10]:
url = GetURL()
url

['https://finance.vietstock.vn/nganh/1-ban-buon.htm',
 'https://finance.vietstock.vn/nganh/2-bao-hiem.htm',
 'https://finance.vietstock.vn/nganh/3-bat-dong-san.htm',
 'https://finance.vietstock.vn/nganh/5-chung-khoan.htm',
 'https://finance.vietstock.vn/nganh/6-cong-nghe-va-thong-tin.htm',
 'https://finance.vietstock.vn/nganh/7-ban-le.htm',
 'https://finance.vietstock.vn/nganh/8-cham-soc-suc-khoe.htm',
 'https://finance.vietstock.vn/nganh/10-khai-khoang.htm',
 'https://finance.vietstock.vn/nganh/11-ngan-hang.htm',
 'https://finance.vietstock.vn/nganh/12-nong-lam-ngu.htm',
 'https://finance.vietstock.vn/nganh/15-sx-thiet-bi-may-moc.htm',
 'https://finance.vietstock.vn/nganh/16-sx-hang-gia-dung.htm',
 'https://finance.vietstock.vn/nganh/17-san-pham-cao-su.htm',
 'https://finance.vietstock.vn/nganh/18-sx-nhua-hoa-chat.htm',
 'https://finance.vietstock.vn/nganh/19-thuc-pham-do-uong.htm',
 'https://finance.vietstock.vn/nganh/20-che-bien-thuy-san.htm',
 'https://finance.vietstock.vn/nganh/21

In [11]:
#Get full stocks along with industry
full_stock =[]
for i in url:
  try:
    industry = pd.read_html(requests.get(i,headers=headers).text)
    industry = pd.DataFrame(industry[1])
    full_stock.append(industry)
  except:
    pass
symbols = pd.concat(full_stock, axis=0, ignore_index=True)
symbols['Giá 1 ngày'] = symbols['Giá 1 ngày'].map(lambda x: x.split(' ')[0].replace(',',''))
symbols['Giá 1 ngày'] = symbols['Giá 1 ngày'].astype(int)
symbols = symbols.drop(columns=['STT','Sàn'])
symbols.rename(columns = {'Mã CK':'Stock', 'Ngành cấp 3':'Sector',
               'KLCPLH':'Share numbers', 'Giá 1 ngày':'Real-time price','Giá 5 ngày': 'Average 5-day price'}, inplace = True)
symbols.tail()

,Stock,Sector,Share numbers,Real-time price,Average 5-day price
735,VCM,Dịch vụ việc làm,3000000,15000,16500.0
736,VLA,Dịch vụ lập trình máy tính,1080000,17500,18000.0
737,VNC,"Các dịch vụ chuyên môn, khoa học và kỹ thuật khác",10499560,33300,32000.0
738,OGC,Các hoạt động đầu tư tài chính khác,300000000,6300,7100.0
739,TVC,Các hoạt động đầu tư tài chính khác,103610670,16800,17500.0


In [12]:
#Cách khác
#Get real time price for stock
def getrealtimeprice(stock):
  r = requests.get('https://finance.vietstock.vn/{}/thong-ke-giao-dich.htm'.format(stock),headers=headers)
  soup = bs(r.content)
  price = soup.select('h2#stockprice span')[0].text
  price = price.replace(',', '')
  return int(price)

#Financial ratio analysis
Create function to calculate financial ratio for stock list

In [13]:
stocklist=[] 
def FinRatio(stocklist):
  FinRatio = {}
  for stock in stocklist:
    BS = Financialdata(stock,start_date,end_date).get_BS()
    IS = Financialdata(stock,start_date,end_date).get_IS()
    CF = Financialdata(stock,start_date,end_date).get_CF()
    price = int(symbols.loc[symbols['Stock'] == stock, 'Real-time price'].to_string(index=False).strip())
    KLCPLH = int(symbols.loc[symbols['Stock'] == stock, 'Share numbers'].to_string(index=False).strip())
    market_cap = KLCPLH * price

    FinRatio[stock] = {}
    FinRatio[stock]['market_cap'] = market_cap
    FinRatio[stock]['Trailing PE'] = market_cap/(IS.loc['Lợi nhuận sau thuế thu nhập doanh nghiệp'][0:4].sum())
    FinRatio[stock]['Price/FCF'] = market_cap/ (CF.loc['Lưu chuyển tiền thuần từ hoạt động kinh doanh'][0]+CF.loc['Mua sắm TSCĐ'][0]+CF.loc['Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác'][0])
    FinRatio[stock]['ROE'] = IS.loc['Doanh thu thuần'].mean()/BS.loc['Vốn chủ sở hữu'].mean()
    FinRatio[stock]['ROA'] = IS.loc['Doanh thu thuần'].mean()/BS.loc['TỔNG CỘNG TÀI SẢN'].mean()
    FinRatio[stock]['EPS 4 latest quart'] = IS.loc['Lãi cơ bản trên cổ phiếu'][0:4].sum()
    FinRatio[stock]['Operating margin'] = IS.loc['Lợi nhuận thuần từ hoạt động kinh doanh'][0:4].mean()/IS.loc['Doanh thu thuần'][0:4].mean()
    FinRatio[stock]['Operating cashflow margin'] = CF.loc['Lưu chuyển tiền thuần từ hoạt động kinh doanh'][0:4].mean()/IS.loc['Doanh thu thuần'][0:4].mean()

  df = pd.DataFrame.from_dict(FinRatio,orient='index')
  return df

In [14]:
#CHú ý: Dành cho các tổ chức tài chính
stocklist=[] 
def FinRatio_finance(stocklist):
  FinRatio = {}
  for stock in stocklist:
    BS = Financialdata(stock,start_date,end_date).get_BS()
    IS = Financialdata(stock,start_date,end_date).get_IS()
    CF = Financialdata(stock,start_date,end_date).get_CF()
    price = int(symbols.loc[symbols['Stock'] == stock, 'Real-time price'].to_string(index=False).strip())
    KLCPLH = int(symbols.loc[symbols['Stock'] == stock, 'Share numbers'].to_string(index=False).strip())
    market_cap = KLCPLH * price

    FinRatio[stock] = {}
    FinRatio[stock]['market_cap'] = market_cap
    FinRatio[stock]['Trailing PE'] = market_cap/(IS.loc['Lợi nhuận sau thuế thu nhập doanh nghiệp'][0:4].sum())
    FinRatio[stock]['ROE'] = IS.loc['Thu nhập lãi thuần'].mean()/BS.loc['Vốn chủ sở hữu'].mean()
    FinRatio[stock]['ROA'] = IS.loc['Thu nhập lãi thuần'].mean()/BS.loc['TỔNG CỘNG NGUỒN VỐN'].mean()
    FinRatio[stock]['EPS 4 latest quart'] = IS.loc['Lãi cơ bản trên cổ phiếu'][0:4].sum()

  df = pd.DataFrame.from_dict(FinRatio,orient='index')
  return df


##Portfolio analysis

In [15]:
portfolio = ['VNM','VIC']
FinRatio(portfolio)

,market_cap,Trailing PE,Price/FCF,ROE,ROA,EPS 4 latest quart,Operating margin,Operating cashflow margin
VNM,187260007872000,17.285173,70.215299,0.461767,0.313625,4572,0.221353,0.160257
VIC,329912122379700,74.041451,-179.312638,0.251776,0.080582,476,0.105314,0.090409


##Peers analysis

In [16]:
# Get stock list in the same sector with selected stock
industry = symbols.loc[symbols['Stock'] == stock, 'Sector'].to_string(index=False).strip()
stocklist = symbols['Stock'].loc[symbols['Sector'] == industry].tolist()
# Compare ratios between stocks in the same sector
df = FinRatio(stocklist)

#Historical data

In [63]:
import logging as logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
URL_CAFE = "http://s.cafef.vn/Lich-su-giao-dich-"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

def convert_date(text, date_type = '%Y-%m-%d'):
  return datetime.strptime(text, date_type)
  
def convert_text_dateformat(text, origin_type = '%Y-%m-%d', new_type = '%Y-%m-%d'):
  return convert_date(text, origin_type).strftime(new_type)

def split_change_col(text):
    return re.sub(r'[\(|\)%]', '', text).strip().split()

class DataLoadProto():
    def __init__(self, symbols, start, end, *arg, **karg):
        self.symbols = symbols
        self.start = convert_text_dateformat(start, new_type = '%d/%m/%Y')
        self.end = convert_text_dateformat(end, new_type = '%d/%m/%Y')

class DataLoaderCAFE(DataLoadProto):
    def __init__(self, symbols, start, end, *arg, **karg):
        self.symbols = symbols
        self.start = start
        self.end = end
        super(DataLoaderCAFE, self).__init__(symbols, start, end)

    def download(self):
        stock_datas = []
        if not isinstance(self.symbols, list):
            symbols = [self.symbols]
        else:
            symbols = self.symbols

        for symbol in symbols:
            stock_datas.append(self.download_one(symbol))

        data = pd.concat(stock_datas, axis=1)
        return data

    def download_one(self, symbol):
        stock_data = pd.DataFrame(columns=['date', 'change_perc1', 'change_perc2',
                                           'open', 'high', 'low', 'close',
                                           'avg', 'volume_match', 'volume_reconcile'])

        for i in range(1000):
            stock_slice_batch = self.download_batch(i + 1, symbol)
            stock_data = pd.concat([stock_data, stock_slice_batch], axis=0)
            try:
                date_end_batch = stock_slice_batch.date.values[-1]
            except:
                # start date is holiday or weekend
                break
            is_touch_end = convert_date(self.start, '%d/%m/%Y') == convert_date(date_end_batch, '%d/%m/%Y')
            # logging.info('batch: {}; start date out range: {}; date_end_batch: {}'.format(i + 1, is_touch_end, date_end_batch))
            if is_touch_end:
                break

        stock_data['change_perc1'], stock_data['change_perc2'] = stock_data['change_perc'].apply(split_change_col).str
        if 'change_perc' in stock_data.columns:
            stock_data.pop('change_perc')
        if 'avg' in stock_data.columns:
            stock_data.pop('avg')
            stock_data = stock_data.set_index('date').apply(pd.to_numeric, errors='coerce')
            stock_data.index = list(map(lambda text: convert_date(text, date_type='%d/%m/%Y'), stock_data.index))
            stock_data.index.name = 'date'
            stock_data = stock_data.sort_index()
            stock_data.fillna(0, inplace=True)
            stock_data['volume'] = stock_data.volume_match + stock_data.volume_reconcile

        # Create multiple columns
        #iterables = [stock_data.columns.tolist(), [symbol]]
        #mulindex = pd.MultiIndex.from_product(iterables, names=['Attributes', 'Symbols'])
        #stock_data.columns = mulindex


        logging.info('data {} from {} to {} have already cloned!' \
                     .format(symbol,
                             convert_text_dateformat(self.start, origin_type = '%d/%m/%Y', new_type = '%Y-%m-%d'),
                             convert_text_dateformat(self.end, origin_type='%d/%m/%Y', new_type='%Y-%m-%d')))

        return stock_data

    def download_batch(self, id_batch, symbol):
        form_data = {'ctl00$ContentPlaceHolder1$scriptmanager':'ctl00$ContentPlaceHolder1$ctl03$panelAjax|ctl00$ContentPlaceHolder1$ctl03$pager2',
                       'ctl00$ContentPlaceHolder1$ctl03$txtKeyword':symbol,
                       'ctl00$ContentPlaceHolder1$ctl03$dpkTradeDate1$txtDatePicker':self.start,
                       'ctl00$ContentPlaceHolder1$ctl03$dpkTradeDate2$txtDatePicker':self.end,
                       '__EVENTTARGET':'ctl00$ContentPlaceHolder1$ctl03$pager2',
                       '__EVENTARGUMENT':id_batch,
                       '__ASYNCPOST':'true'}
        url = URL_CAFE+symbol+"-1.chn"
        r = requests.post(url, data = form_data, headers = headers, verify=False)
        soup = bs(r.content, 'html.parser')
        #print(id_batch)
        table = soup.find('table')
        stock_slice_batch = pd.read_html(str(table))[0].iloc[2:, :12]

        stock_slice_batch.columns = ['date', 'adjust', 'close', 'change_perc', 'avg',
                        'volume_match', 'value_match', 'volume_reconcile', 'value_reconcile',
                        'open', 'high', 'low']

        return stock_slice_batch


#Valuation models with PE
Determine if any stock is under/over valued with PE

In [17]:
#Add current price into data frame
df_price={}
for stock in stocklist:
  df_price[stock] = {}
  df_price[stock]['Current Price'] = int(symbols.loc[symbols['Stock'] == stock, 'Real-time price'].to_string(index=False).strip())
df_price = pd.DataFrame.from_dict(df_price,orient='index')
a = df.join(df_price, how='outer')
#Calculate the mean PE ratio
PE_mean = a['Trailing PE'].mean()
#Calculate the fair market value
a['Fair Market value']= PE_mean * a['EPS 4 latest quart']
#Calculate over/under valued ratio
a['Over_Under ratio'] = a['Current Price']/a['Fair Market value']
#Show a label of under valued and over valued stocks
a['Value_label'] = np.where(a['Over_Under ratio']<1.0, 'Under Valued', 'Fair or Over Valued')
a.insert(0, 'Value_label',a.pop('Value_label'))
a

,Value_label,market_cap,Trailing PE,Price/FCF,ROE,ROA,EPS 4 latest quart,Operating margin,Operating cashflow margin,Current Price,Fair Market value,Over_Under ratio
AGG,Under Valued,3504486935950,7.814936,5.058103,0.154683,0.038605,5093,0.203328,0.699587,42350,164048.733827,0.258155
API,Under Valued,1639020000000,28.599151,-20.842567,0.256949,0.051637,1618,0.119954,-0.142817,46300,52116.797827,0.888389
BCE,Under Valued,493500000000,9.603316,6.040376,0.318050,0.120149,1390,0.140351,0.453116,14100,44772.774400,0.314924
CDC,Under Valued,323234125200,17.347357,-2.666691,0.378840,0.118297,917,0.051480,-0.238835,14700,29537.146852,0.497678
CEO,Fair or Over Valued,2573399850000,-16.279716,-320.571298,0.187452,0.078589,0,-0.031239,-0.189270,10000,0.000000,inf
CIG,Fair or Over Valued,233080208330,-8.239077,717.678461,0.012321,0.003753,0,-13.935814,-0.197606,7390,0.000000,inf
CKG,Under Valued,2012994656400,14.268417,-10.052654,0.398821,0.063262,2558,0.153651,0.204443,24400,82394.789148,0.296135
D11,Under Valued,176247858500,5.425097,9.089401,0.581124,0.104398,3960,0.214427,-0.522976,26900,127554.091097,0.210891
D2D,Fair or Over Valued,1594030270800,7.037725,-41.312290,0.160793,0.062652,0,0.723423,-0.736308,52600,0.000000,inf
DIG,Fair or Over Valued,13171619822000,19.383432,-70.761789,0.139612,0.063821,0,0.119557,0.688917,31000,0.000000,inf


In [18]:
#Show a list of stocks that are considered under valued and P/FCF > 0
a[(a['Value_label'] == 'Under Valued')&(a['Price/FCF']>0)].index

Index(['AGG', 'BCE', 'D11', 'DXG', 'FIR', 'HLD', 'HUT', 'IJC', 'NTL', 'PDR',
       'TDC', 'VHM', 'VPI'],
      dtype='object')

#Ben Graham Stock Screening
This method of Ben Graham is often used very effectively when the market is in a bear trend or the stock is under accumulation.

Phương pháp này của Ben Graham thường sử dụng rất hiệu quả trong 1 thị trường giá xuống hoặc thị trường tích lũy ở vùng đáy.

<b> Chú ý: Bộ lọc theo phương pháp của Ben Graham không áp dụng cho cổ phiếu thuộc lĩnh vực tài chính như ngân hàng, công ty bảo hiểm hay công ty chứng khoán.



In [28]:
stocklist=[] 
def BenGrahamscreening(stocklist):
  FinRatio = {}
  for stock in stocklist:
    BS = Financialdata(stock,start_date,end_date).get_BS()

    price = int(symbols.loc[symbols['Stock'] == stock, 'Real-time price'].to_string(index=False).strip())
    KLCPLH = int(symbols.loc[symbols['Stock'] == stock, 'Share numbers'].to_string(index=False).strip())
    market_cap = KLCPLH * price
    Current_asset = BS_data.loc['TỔNG CỘNG TÀI SẢN'][0:4].mean()-BS_data.loc['Tài sản dài hạn'][0:4].mean()
    Total_liabilities = BS.loc['Nợ phải trả'][0:4].mean()
    NCAV = Current_asset - Total_liabilities
    NNWC = BS.loc['Tiền và các khoản tương đương tiền'][0:4].mean()+BS.loc['Các khoản đầu tư tài chính ngắn hạn'][0:4].mean() + 0.75 * BS.loc['Các khoản phải thu ngắn hạn'][0:4].mean() + 0.5 * BS.loc['Hàng tồn kho'][0:4].mean() - Total_liabilities

    FinRatio[stock] = {}
    FinRatio[stock]['P_NCAV'] = market_cap /  NCAV
    FinRatio[stock]['P_NNWC'] = market_cap / NNWC

  df = pd.DataFrame.from_dict(FinRatio,orient='index')
  return df

In [53]:
start_date = '2019-01-01'
end_date = str(datetime.now().strftime('%Y-%m-%d'))
stock = 'MSN'
BS_data = Financialdata(stock,start_date,end_date).get_BS()
price = int(symbols.loc[symbols['Stock'] == stock, 'Real-time price'].to_string(index=False).strip())
KLCPLH = int(symbols.loc[symbols['Stock'] == stock, 'Share numbers'].to_string(index=False).strip())
market_cap = price*KLCPLH
Current_asset = BS_data.loc['TỔNG CỘNG TÀI SẢN'][0:4].mean()-BS_data.loc['Tài sản dài hạn'][0:4].mean()
Long_term_asset = BS_data.loc['Tài sản dài hạn'][0:4].mean()
Total_liabilities = BS_data.loc['Nợ phải trả'][0:4].mean()
NCAV = Current_asset - Total_liabilities
NNWC = BS_data.loc['Tiền và các khoản tương đương tiền'][0:4].mean()+BS_data.loc['Các khoản đầu tư tài chính ngắn hạn'][0:4].mean()+ 0.75 * BS_data.loc['Các khoản phải thu ngắn hạn'][0:4].mean()+ 0.5 * BS_data.loc['Hàng tồn kho'][0:4].mean() - Total_liabilities
P_NCAV = market_cap /  NCAV
P_NNWC = market_cap / NNWC
print(P_NCAV)
print(P_NNWC)

-2.88941818883039
-2.460440860221898


In [64]:
Historical = DataLoaderCAFE(stock,start_date,end_date).download()
#volume = Historical.loc['volume'][0:66].mean()
Historical

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
2021-09-29 18:33:17,327 : INFO : data MSN from 2019-01-01 to 2021-09-29 have already cloned!


,change_perc1,change_perc2,open,high,low,close,volume_match,volume_reconcile,adjust,value_match,value_reconcile,volume
date,,,,,,,,,,,,
2019-01-02,1.1,1.42,78.6,78.9,78.2,78.6,433670,66000,77.01,34045000000,5280000000,499670
2019-01-03,-0.6,-0.76,78.9,78.9,77.6,78.0,565640,0,76.42,44161000000,0,565640
2019-01-04,-1.8,-2.31,77.0,77.5,75.5,76.2,463070,178860,74.66,35389000000,13681081200,641930
2019-01-07,2.0,2.62,78.1,78.5,77.2,78.2,336380,217130,76.62,26223000000,17164583500,553510
2019-01-08,0.8,1.02,78.2,79.1,78.1,79.0,279810,177130,77.40,22032000000,13952444900,456940
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-23,-2.0,-1.36,146.5,148.5,143.0,145.0,932400,0,145.00,136149000000,0,932400
2021-09-24,-2.6,-1.79,145.0,145.5,141.8,142.4,1161000,0,142.40,166187000000,0,1161000
2021-09-27,-7.4,-5.20,142.7,143.5,135.0,135.0,1213100,0,135.00,169065000000,0,1213100


In [67]:
Historical['value'] = Historical['volume']*Historical['close']
volume = Historical['value'][0:66].mean()
volume

152668830.61363637

In [ ]:
# Get stock list in the same sector with selected stock
industry = symbols.loc[symbols['Stock'] == stock, 'Sector'].to_string(index=False).strip()
stocklist = symbols['Stock'].loc[symbols['Sector'] == industry].tolist()
b = BenGrahamscreening(stocklist)
b

Stock screening according to NCAV:
> NCAV = Current Assets – Total Liabilities

- 1. P/NCAV < 1
- 2. 3M Average Volume > 500 million VND
- 3. Arranging the P/NCAV ratio in ascending order

Stock screening according to NNWC:
> NNWC = Cash & Cash equivalents + Short-term investment + 0.75 x Account Receivables + 0.5 x Inventory – Total Liabilities

- 1. P/NNWC < 1
- 2. 3M Average Volume > 500 million VND
- 3. Arranging the P/NNWC ratio in ascending order

#Warren Buffett Stock Screening
Need data on all stocks in Vietnam market
- 1. Market capitalization > 50 million USD
- 2. Operating income > 0 for 5 consecutive years
- 3. ROE greater than 15% in the last 2 years and accumulated for 12 months
- 4. P/FCF ratio (Price/Free Cash flow) is among the lowest 30% in the whole market
- 5. Operating income margin > Operating income margin Median
- 6. Net income margin > Net income margin Median

#Philip Fisher Stock Screening
Tập trung vào những cổ phiếu tăng trưởng
- 1. Sales 5Y CAGR > Sales 5Y CAGR Median
- 2. Doanh thu tăng trưởng trong 2 năm gần nhất
- 3. Net income margin > Net income margin Median
- 4. Tỷ lệ PEG (5Y Growth) < 0.5
- 5. Sắp xếp tỷ lệ PEG theo thứ tự tăng dần


#CANSLIM Stock Screening
- C: Current Quaterly Earnings Per Share:
  
  + Tăng trưởng EPS quý gần nhất và quý gần liền kề phải đạt tối thiểu 20% – 25% so với quý cùng kỳ

- 